#### Now create a multiclass problem 
Here we will include all classes so basically we have to load the data again, it will almost be the same code as for binary classification

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd, norm
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from PIL import Image
import ast
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD
from sklearn import datasets, svm, metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
#Import naive bayes
from sklearn.naive_bayes import GaussianNB
#import Knn
from sklearn.neighbors import KNeighborsClassifier
#Import confusion matrixdisplay
from sklearn.metrics import ConfusionMatrixDisplay
#For truncated svd, we need to import the base estimator and classifier mixin
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

#For multilabel classification
from sklearn.multioutput import MultiOutputClassifier


In [11]:
def load_image(path):
    img = Image.open(f'{base_dir}{path}')
    #Resize the image to 320x320
    img = img.resize((320, 320))
    img = np.array(img)
    img = np.array(img)
    img = img.flatten()
    #img = img.reshape((-1,1))
    return img

def one_hot_encode(label):
    # One hot encoding
    place_holder = [0, 0, 0, 0, 0]
    if 'CrackA' in label:
        place_holder[0] = 1
    if 'CrackB' in label:
        place_holder[1] = 1
    if 'CrackC' in label:
        place_holder[2] = 1
    if 'FingerFailure' in label:
        place_holder[3] = 1
    if 'Negative' in label:
        place_holder[4] = 1
    return place_holder

def get_images(data, M=102,train=True):
    # Load the data such that the collumns of X is the image as a vector
    # Initialize np array of size (320*320,len(data))
    X = [] #np.zeros(((320 * 320),M*2))
    y = []#[None] * (M*2)
    neg_count = 0
    pos_count = 0
    
    for (i, row) in tqdm(data.iterrows()):
        #Get the label
        label = ast.literal_eval(row['Label'])
        lab_encoding = one_hot_encode(label)
        
        if lab_encoding[4] == 1 and neg_count < M:
            im = load_image(row['ImageDir'])
            X.append(im)
            y.append(lab_encoding)
            if train:
                neg_count += 1
        elif not lab_encoding[4]:
            im = load_image(row['ImageDir'])
            X.append(im)
            y.append(lab_encoding)
            if train:
                pos_count += 1
        elif neg_count >= M and pos_count >= M:
            print("What")
            break
        else:
            continue
    X = np.array(X)
    X = X.T #We want the collumns to be the images
    return X, np.array(y)

In [12]:
# Load the data
train_data = pd.read_csv('/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/Train.csv')
train_data = train_data.drop(['Unnamed: 0'], axis=1)
test_data = pd.read_csv('/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/VitusData/Val.csv')
test_data = test_data.drop(['Unnamed: 0'], axis=1)
base_dir = '/Users/madsandersen/PycharmProjects/BscProjektData/BachelorProject/Data/'

# Training data size
M = 60 # Number of images

In [13]:
#Create the X and y matrices
X_train,y_train = get_images(train_data,train=True)
X_test,y_test = get_images(test_data,train=False)

8879it [00:01, 7889.20it/s] 


What


7211it [00:27, 262.48it/s]


(102400, 204)

### Multilabel classification
Fit at binary classifier to each class to test if they belong to it

In [ ]:
#clf = MultiOutputClassifier(svm.SVC(kernel='linear', C=0.01)).fit(X_train.T, y_train)
clf = MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, random_state=0)).fit(X_train.T, y_train)

In [ ]:
preds = clf.predict(X_test.T)

In [ ]:
binary_predictions = {
                       'predA':preds[:,0],
                       'predB':preds[:,1],
                       'predC':preds[:,2],
                       'predFF': preds[:,3],
                       'predNeg':preds[:,4]
                    }

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(5,15))

for idx,lab in enumerate(binary_predictions.keys()):
    y_hat = binary_predictions[lab]
    disp = ConfusionMatrixDisplay.from_predictions(y_test[:,idx], y_hat)
    disp.plot(ax=axes.flat[idx])